### Post Operation

In [3]:
''' There are a lot of tasks that must perform a 'post operation'.
        e.g. close(), delete(), reset()
Unless the results would be devastating.
Ex. Most of IO streams, e.g. file and socket must be properly closed. '''
f = open('./files/hello.txt', 'r')
print(f.read())  #  If an exception occurs here the file would not be closed.
f.close()

Hello how do you do?


#### Problem of File Closing

In [11]:
''' Acessing files should be in a 'try' block since a lot of things could go wrong
  and close() should be in the 'finally' block to ensure that it would be executed.
Closing a None reference is an error, we need to check the file reference is not None. '''
try:
    f = open('./files/hello.txt','r')
    print(f.read())
except Exception as e:
    print(e)
finally:
    if f is not None:
        f.close()

Hello how do you do?


### Context Manager

In [10]:
''' Pyton provides 'with' statement that supports Context Manager.
             with <expression> as <reference>:
                 <body>
A context manager sets up a scope where an object can be created and used
then at end of the scope the object is automatically closed.

<expression> is executed to create an object that is referred to by the
<reference> which can be used in the <body>. 
Then at the end of the <body> the object's close() will be executed.
'as' is optional, in case the object does not needed to be referred to.

'with' statement does not need to be in 'try-except' but in cases of
file opening, it should.  '''
try:
    with open('./files/hello.txt', 'r') as f:
        print(f.read())
except Exception as e:
    print(e)

Hello how do you do?


In [13]:
''' The object created by <expression> is called a 'context manager'.
A context manager creates an object to be managed called 'managed object'.
A context manager must have:
   __enter__() to create and return a managed object to the <reference>.
   __exit__() to delete or close the managed object for post operation.

When the <body> terminates, the context manager's __exit__() is invoked.

To show how a context manager work we will create one manually.
But for simplicity,  let a 'Hello' stre be the managed object. '''
class ctx_mgr:
    def __enter__(self):
        print('enter', end=', ')
        return 'Hello'
    def  __exit__(self, exc_type, exc_value, traceback):
        print('exit', end=', ')
        
with ctx_mgr() as t:
    print(t, end=', ')

enter, Hello, exit, 

In [15]:
''' Now we create managed object manually. '''
class mg_obj:
    def __init__(self, ctx):  ##  ctx is the context manager.
        self.ctx = ctx
        print('init', end=',')
    def do_something(self):
        print('do_something', end=',')
    def __del__(self):  ##  __del__() is executed when the object is deleted.
        print('del', end=',')
        
class mg_obj_ctx_mgr:
    def __enter__(self):
        print('enter', end=',')
        self.ins = mg_obj(self)     # managed object is created.
        return self.ins
    def __exit__(self, exc_type, exc_val, exc_tb):
        print('exit', end=',')

with mg_obj_ctx_mgr() as mo:
    mo.do_something()

enter,init,do_something,exit,

In [17]:
''' File Context Manager:
Python has 'TextIOWrapper' for file context manager that is created by open().
     __enter__() returns the opened file.
     __exit__() closes the file.   '''

class file_ctx_mgr():
    def __init__(self, fname):
        self.fname = fname
    def __enter__(self):
        self.file = open(self.fname, 'r')
        return self.file
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

with file_ctx_mgr('./files/hello.txt') as f:
    print(f.read())

Hello how do you do?


In [18]:
''' Creating a function is easier than a class.
We can to create a context managers from a coroutine function.
The function must be decorated with @contextlib.contextmanager.
The function must have a 'yield' that produces whatever the __enter__() return.
  The code before 'yield' will be executed as in __enter__().
  The code after 'yield' will be executed as in __exit__().   '''
import contextlib

@contextlib.contextmanager
def file_mgr(name):
    try:
        f = open(name, 'r')
        yield f
    finally:
        f.close()

with file_mgr('./files/hello.txt') as f:
    print(f.read())

Hello how do you do?


In [19]:
''' If the managed object returns a result.
We can combine the managed object and the context manager into a coroutine. '''
@contextlib.contextmanager
def hello(name):
    print('enter' , end=', ')   #  Do what __enter__() do
    yield "Hello " + name       #  Return a result
    print('exit' , end=', ')    #  Do what __exit__() do

with hello('John') as result:
    print(result, end=', ')

enter, Hello John, exit, 

In [20]:
''' In case we already have a class for managed object with
     __init__() that can be used as __enter__().
     close() that can used as __exit__().
Its context manager can be created with the help of a wrapper.
     contextlib.closing(<managed object>) 
'''
class Hi:
    def __init__(self):
        print('init', end=',')
    def hi(self, name):
        print('Hi '+name, end=',')
    def close(self):
        print ('close', end=',')

with contextlib.closing(Hi()) as h:
    h.hi('John')

init,Hi John,close,

In [21]:
''' Python 3.10 allows multiple context managers in a 'with'. '''
with( open('./files/hello.txt', 'r') as f1, open('./files/hi.txt', 'r') as f2 ):
  print(f1.readline())
  print(f2.readline())

Hello how do you do?
Hi how are you?
